# Исследование способов определения ортогонального поворота документов
- Автор: Кирилл Киселев
- Дата начала: 21.06.2022
- Описание исследования: оптимизация способы определения ортогонального положения документов.

## 1. Имеющиеся изображения

## 2. Загрузка библиотек

In [1]:
import cv2
import imutils
import numpy as np
import matplotlib as plt
import os
import math
from PIL import Image
from pyzbar import pyzbar

import pytesseract
from pytesseract import Output

import re

## 3. Глобальные переменные

In [35]:
!pwd

/home/kirillk/PycharmProjects/useful_notebooks


In [34]:
os.listdir('..')

['rec_to_image',
 'ocr_data_generation',
 'barcode_scaner',
 'ocr-benchmark',
 'distance_calculation',
 'ocr_data_engineering',
 'ocr_ortho_angle_def',
 'text_generator',
 'paddle_ocr',
 'cut_videos',
 'image_markup',
 'data_engineering',
 'DeepFace_test',
 'deepface-master',
 'useful_notebooks',
 'pythonProject']

In [16]:
FILE_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases_6/'

## 4. Необходимые функции

In [17]:
def detect_shape(contour):
    """Returns the shape (e.g. 'triangle', 'square') from the contour"""

    detected_shape = '-----'

    # Calculate perimeter of the contour:
    perimeter = cv2.arcLength(contour, True)

    # Get a contour approximation:
    contour_approx = cv2.approxPolyDP(contour, 0.03 * perimeter, True)

    # Check if the number of vertices is 3. In this case, the contour is a triangle
    if len(contour_approx) == 3:
        detected_shape = 'triangle'

    # Check if the number of vertices is 4. In this case, the contour is a square/rectangle
    elif len(contour_approx) == 4:

        # We calculate the aspect ration from the bounding rect:
        x, y, width, height = cv2.boundingRect(contour_approx)
        aspect_ratio = float(width) / height

        # A square has an aspect ratio close to 1 (comparison chaining is used):
        if 0.90 < aspect_ratio < 1.10:
            detected_shape = "square"
        else:
            detected_shape = "rectangle"

    # Check if the number of vertices is 5. In this case, the contour is a pentagon
    elif len(contour_approx) == 5:
        detected_shape = "pentagon"

    # Check if the number of vertices is 6. In this case, the contour is a hexagon
    elif len(contour_approx) == 6:
        detected_shape = "hexagon"

    # The shape as more than 6 vertices. In this example, we assume that is a circle
    else:
        detected_shape = "circle"

    # return the name of the shape and the found vertices
    return detected_shape, contour_approx

In [18]:
def find_contour(gray_image, threshold=160):
    _, thresh = cv2.threshold(gray_image, threshold, 255, 0)  # 160
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    max_contour = 0
    num_of_contour = None

    for num, i in enumerate(contours):
        if len(i) > max_contour:
            max_contour = len(i)
            num_of_contour = num
    return contours[num_of_contour]    

## 5. Данные

### Оптимизация скрипта поворота фото 

In [19]:
%%time
im = cv2.imread(FILE_PATH_2 + rotated_photo_list[0])
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

doc_contour = find_contour(im_gray)
shape, vertices = detect_shape(doc_contour)

assert shape=='rectangle'

x1, y1 = vertices[1][0][0], vertices[1][0][1]
x2, y2 = vertices[2][0][0], vertices[2][0][1]
xM, yM = vertices[1][0][0], vertices[2][0][1]

# catet_M_2 = abs(x2 - xM)
# catet_1_M = abs(yM - y1)

catet_M_2 = (x2 - xM)
catet_1_M = (yM - y1)

tg_a = catet_1_M / catet_M_2
angle = round(math.degrees(math.atan(tg_a)))

# здесь осуществили поворот, rotate_bound поворачивает изображение по часовой стрелке
img = imutils.rotate_bound(im_gray, -angle)

# снова ищем контуры документа
contour_for_crop = find_contour(img)
_, vertices_2 = detect_shape(contour_for_crop)

x_1, x_2 = min([vertices_2[0][0][0], vertices_2[1][0][0]]), max([vertices_2[2][0][0], vertices_2[3][0][0]])
y_1, y_2 = min([vertices_2[0][0][1], vertices_2[3][0][1]]), max([vertices_2[2][0][1], vertices_2[2][0][1]])

im = imutils.rotate_bound(im, -angle)
im = im[y_1:y_2, x_1:x_2]
im = cv2.resize(im, (0, 0), fx=0.3, fy=0.3)

cv2.imshow("Image", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'FILE_PATH_2' is not defined

# Current research point

In [20]:
%%time
im = cv2.imread(FILE_PATH + photo_list[8])
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

doc_contour = find_contour(im_gray)
shape, vertices = detect_shape(doc_contour)

assert shape=='rectangle'

x1, y1 = vertices[1][0][0], vertices[1][0][1]
x2, y2 = vertices[2][0][0], vertices[2][0][1]
xM, yM = vertices[1][0][0], vertices[2][0][1]

# catet_M_2 = abs(x2 - xM)
# catet_1_M = abs(yM - y1)

catet_M_2 = (x2 - xM)
catet_1_M = (yM - y1)

tg_a = catet_1_M / catet_M_2
angle = round(math.degrees(math.atan(tg_a)))

# здесь осуществили поворот, rotate_bound поворачивает изображение по часовой стрелке
img = imutils.rotate_bound(im_gray, -angle)

# снова ищем контуры документа
contour_for_crop = find_contour(img)
_, vertices_2 = detect_shape(contour_for_crop)

x_1, x_2 = min([vertices_2[0][0][0], vertices_2[1][0][0]]), max([vertices_2[2][0][0], vertices_2[3][0][0]])
y_1, y_2 = min([vertices_2[0][0][1], vertices_2[3][0][1]]), max([vertices_2[2][0][1], vertices_2[2][0][1]])

im = imutils.rotate_bound(im, -angle)
im = im[y_1:y_2, x_1:x_2]
# im = cv2.resize(im, (0, 0), fx=0.3, fy=0.3)

cv2.imwrite('/home/kirillk/datasets/OCR/ocr_test_cases_3/000.jpg', im)

cv2.imshow("Image", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

CPU times: user 771 ms, sys: 199 ms, total: 969 ms
Wall time: 2.81 s


In [21]:
FILE_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases_3/'

In [22]:
# список фото
photo_list = sorted([photo for photo in os.listdir(FILE_PATH) if photo.endswith('.JPG')])
photo_list

['001.JPG',
 '002.JPG',
 '003.JPG',
 '004.JPG',
 '005.JPG',
 '006.JPG',
 '007.JPG',
 '008.JPG',
 '009.JPG',
 '010.JPG',
 '011.JPG',
 '012.JPG',
 '013.JPG']

In [23]:
photo_list_png = sorted([photo for photo in os.listdir(FILE_PATH) if photo.endswith('.png')])
photo_list_png

['013.png', '014.png']

In [24]:
FILE_PATH_1 = '/home/kirillk/datasets/OCR/ocr_test_cases/test_cases/'

In [25]:
# список фото
photo_list_1 = sorted([photo for photo in os.listdir(FILE_PATH_1) if photo.endswith('.jpg')])
photo_list_1

['000.jpg',
 '001.jpg',
 '002.jpg',
 '003.jpg',
 '004.jpg',
 '005.jpg',
 '006.jpg',
 '007.jpg',
 '008.jpg',
 '009.jpg',
 '010.jpg',
 '011.jpg',
 '012.jpg',
 '013.jpg',
 '014.jpg',
 '015.jpg',
 '016.jpg',
 '017.jpg',
 '018.jpg',
 '019.jpg',
 '020.jpg',
 '021.jpg',
 '022.jpg',
 '023.jpg',
 '024.jpg',
 '025.jpg',
 '026.jpg',
 '027.jpg',
 '028.jpg',
 '029.jpg',
 '030.jpg',
 '031.jpg',
 '032.jpg',
 '033.jpg',
 '034.jpg',
 '035.jpg',
 '036.jpg',
 '037.jpg',
 '038.jpg',
 '039.jpg',
 '040.jpg',
 '041.jpg',
 '042.jpg',
 '043.jpg',
 '044.jpg',
 '045.jpg',
 '046.jpg',
 '047.jpg',
 '048.jpg',
 '049.jpg',
 '050.jpg',
 '051.jpg',
 '052.jpg',
 '053.jpg',
 '054.jpg',
 '055.jpg',
 '056.jpg',
 '057.jpg',
 '058.jpg',
 '059.jpg',
 '060.jpg',
 '061.jpg',
 '062.jpg',
 '063.jpg',
 '064.jpg',
 '065.jpg',
 '066.jpg',
 '067.jpg',
 '068.jpg',
 '069.jpg',
 '070.jpg',
 '071.jpg',
 '072.jpg',
 '073.jpg',
 '074.jpg',
 '075.jpg',
 '076.jpg',
 '077.jpg',
 '078.jpg',
 '079.jpg',
 '080.jpg',
 '081.jpg',
 '082.jpg',
 '08

In [13]:
%%time
for photo in photo_list_1:
    try:
        print(f'scan name {photo}')
        print(pytesseract.image_to_osd(FILE_PATH_1 + photo, output_type=Output.DICT))
    except:
        print(f'error at scan {photo}')

scan name 000.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 20.34, 'script': 'Cyrillic', 'script_conf': 7.97}
scan name 001.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 19.97, 'script': 'Cyrillic', 'script_conf': 9.07}
scan name 002.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 19.44, 'script': 'Cyrillic', 'script_conf': 5.64}
scan name 003.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 16.24, 'script': 'Cyrillic', 'script_conf': 4.23}
scan name 004.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 21.25, 'script': 'Cyrillic', 'script_conf': 6.13}
scan name 005.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 18.83, 'script': 'Cyrillic', 'script_conf': 4.4}
scan name 006.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 20.02, 'script': 'Cyrillic', 'script_conf': 1.57}
scan name 007.jpg
{'page_num': 0, 'orientation': 0, 'rot

{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 18.32, 'script': 'Cyrillic', 'script_conf': 4.58}
scan name 062.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 20.34, 'script': 'Cyrillic', 'script_conf': 8.33}
scan name 063.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 17.7, 'script': 'Cyrillic', 'script_conf': 6.67}
scan name 064.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 16.37, 'script': 'Cyrillic', 'script_conf': 7.5}
scan name 065.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 17.98, 'script': 'Cyrillic', 'script_conf': 5.65}
scan name 066.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 17.48, 'script': 'Cyrillic', 'script_conf': 6.38}
scan name 067.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 18.38, 'script': 'Cyrillic', 'script_conf': 5.38}
scan name 068.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientati

{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 7.02, 'script': 'Cyrillic', 'script_conf': 1.78}
scan name 123.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 8.27, 'script': 'Cyrillic', 'script_conf': 1.25}
scan name 124.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 10.07, 'script': 'Cyrillic', 'script_conf': 0.27}
scan name 125.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 7.6, 'script': 'Cyrillic', 'script_conf': 1.16}
scan name 126.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 8.82, 'script': 'Cyrillic', 'script_conf': 0.32}
scan name 127.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 9.05, 'script': 'Cyrillic', 'script_conf': 3.33}
scan name 128.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 10.53, 'script': 'Cyrillic', 'script_conf': 1.11}
scan name 129.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_c

{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 13.23, 'script': 'Cyrillic', 'script_conf': 3.01}
scan name 185.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 7.62, 'script': 'Cyrillic', 'script_conf': 3.08}
scan name 186.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 13.7, 'script': 'Cyrillic', 'script_conf': 5.4}
scan name 187.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 10.65, 'script': 'Cyrillic', 'script_conf': 0.38}
scan name 188.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 8.8, 'script': 'Cyrillic', 'script_conf': 1.95}
scan name 189.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 11.8, 'script': 'Cyrillic', 'script_conf': 1.31}
scan name 190.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_conf': 13.3, 'script': 'Cyrillic', 'script_conf': 4.31}
scan name 191.jpg
{'page_num': 0, 'orientation': 0, 'rotate': 0, 'orientation_co

In [26]:
FILE_PATH_1 + photo_list_1[0]

'/home/kirillk/datasets/OCR/ocr_test_cases/test_cases/000.jpg'

In [29]:
%%time
pytesseract.image_to_osd('/home/kirillk/datasets/OCR/ocr_test_cases/test_cases/000.jpg', output_type=Output.DICT)

CPU times: user 1.86 ms, sys: 7.71 ms, total: 9.56 ms
Wall time: 2.23 s


{'page_num': 0,
 'orientation': 0,
 'rotate': 0,
 'orientation_conf': 20.34,
 'script': 'Cyrillic',
 'script_conf': 7.97}

In [30]:
imw = cv2.imread('/home/kirillk/datasets/OCR/ocr_test_cases_3/000.jpg') #, cv2.IMREAD_COLOR)
# img = cv2.resize(im, (0, 0), fx=0.3, fy=0.3)
cv2.imshow("Image", imw)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [31]:
newdata = pytesseract.image_to_osd(imw) #, config='--psm 0 -c min_characters_to_try=100')
newdata

TesseractError: (1, 'UZN file /tmp/tess_p7kf2hew loaded. Estimating resolution as 272 UZN file /tmp/tess_p7kf2hew loaded. Warning. Invalid resolution 0 dpi. Using 70 instead. Too few characters. Skipping this page Error during processing.')

In [ ]:
im = cv2.imread(FILE_PATH_1 + photo_list_1[0], cv2.IMREAD_COLOR)
newdata = tes.image_to_osd(im)

In [ ]:
im_png = cv2.imread(FILE_PATH + photo_list_png[0])
im_png = cv2.cvtColor(im_png, cv2.COLOR_BGR2RGB)

print(osd = pytesseract.image_to_osd(im_png, output_type=Output.DICT))

In [89]:
im_png = cv2.imread(FILE_PATH + photo_list_png[0])
im_png = cv2.cvtColor(im_png, cv2.COLOR_BGR2RGB)

print(osd = pytesseract.image_to_osd(im_png, output_type=Output.DICT))

TesseractError: (1, 'UZN file /tmp/tess_8l1m0p8y loaded. Estimating resolution as 111 UZN file /tmp/tess_8l1m0p8y loaded. Warning. Invalid resolution 0 dpi. Using 70 instead. Too few characters. Skipping this page Error during processing.')

In [77]:
im = cv2.imread(FILE_PATH + photo_list[0])
# im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
# cv2.imshow("Image", im)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [78]:
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
# img_new = Image.fromarray(im)

In [82]:
img_rgb = Image.frombytes('RGB', im.shape[:2], im, 'raw', 'BGR', 0, 0)
print(osd = pytesseract.image_to_osd(img_rgb, output_type=Output.DICT))

TesseractError: (1, 'UZN file /tmp/tess_s914prpu loaded. Estimating resolution as 121 Too few characters. Skipping this page UZN file /tmp/tess_s914prpu loaded. Warning. Invalid resolution 0 dpi. Using 70 instead. Too few characters. Skipping this page Error during processing.')

In [79]:
print(pytesseract.image_to_string(im))

TesseractError: (1, 'Error opening data file /usr/share/tessdata/eng.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'eng\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')

In [71]:
osd = pytesseract.image_to_osd(im, output_type=Output.DICT)
osd

TesseractError: (1, 'UZN file /tmp/tess_kb9eg5ko loaded. Estimating resolution as 304 UZN file /tmp/tess_kb9eg5ko loaded. Warning. Invalid resolution 0 dpi. Using 70 instead. Too few characters. Skipping this page Error during processing.')

In [118]:
osd = pytesseract.image_to_data(FILE_PATH + '013.png', output_type=Output.DICT)
osd

TesseractError: (1, 'Error opening data file /usr/share/tessdata/eng.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'eng\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')

In [105]:
photo_list_1

['000.jpg',
 '001.jpg',
 '002.jpg',
 '003.jpg',
 '004.jpg',
 '005.jpg',
 '006.jpg',
 '007.jpg',
 '008.jpg',
 '009.jpg',
 '010.jpg',
 '011.jpg',
 '012.jpg',
 '013.jpg',
 '014.jpg',
 '015.jpg',
 '016.jpg',
 '017.jpg',
 '018.jpg',
 '019.jpg',
 '020.jpg',
 '021.jpg',
 '022.jpg',
 '023.jpg',
 '024.jpg',
 '025.jpg',
 '026.jpg',
 '027.jpg',
 '028.jpg',
 '029.jpg',
 '030.jpg',
 '031.jpg',
 '032.jpg',
 '033.jpg',
 '034.jpg',
 '035.jpg',
 '036.jpg',
 '037.jpg',
 '038.jpg',
 '039.jpg',
 '040.jpg',
 '041.jpg',
 '042.jpg',
 '043.jpg',
 '044.jpg',
 '045.jpg',
 '046.jpg',
 '047.jpg',
 '048.jpg',
 '049.jpg',
 '050.jpg',
 '051.jpg',
 '052.jpg',
 '053.jpg',
 '054.jpg',
 '055.jpg',
 '056.jpg',
 '057.jpg',
 '058.jpg',
 '059.jpg',
 '060.jpg',
 '061.jpg',
 '062.jpg',
 '063.jpg',
 '064.jpg',
 '065.jpg',
 '066.jpg',
 '067.jpg',
 '068.jpg',
 '069.jpg',
 '070.jpg',
 '071.jpg',
 '072.jpg',
 '073.jpg',
 '074.jpg',
 '075.jpg',
 '076.jpg',
 '077.jpg',
 '078.jpg',
 '079.jpg',
 '080.jpg',
 '081.jpg',
 '082.jpg',
 '08

In [111]:
img = cv2.imread(FILE_PATH_1 + photo_list_1[0])
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [112]:
osd = pytesseract.image_to_osd(photo_list_1, output_type=Output.DICT)
osd

TesseractError: (1, 'UZN file /tmp/tess_ra3b8fy1 loaded. Estimating resolution as 500 UZN file /tmp/tess_ra3b8fy1 loaded. Warning. Invalid resolution 0 dpi. Using 70 instead. Too few characters. Skipping this page Error during processing.')

In [113]:
osd = pytesseract.image_to_osd(FILE_PATH_1 + photo_list_1[0], 
                               output_type=Output.DICT, 
                               config='--psm 0 -c min_characters_to_try=5')
osd

{'page_num': 0,
 'orientation': 0,
 'rotate': 0,
 'orientation_conf': 1.78,
 'script': 'Cyrillic',
 'script_conf': 20.0}